**ANALYSIS QUESTIONS**

1. Discover relationship between Total Complexity Points (as of May 2021) and subsequent PRODUCTION, OUTFLOW, and NET. 9 months is probably preferred, but explore whether there are differences between 3 and 6 months.

2. Same as above, but for each individual Complexity input   (also, including the data on "Other Data" tab as separate variables).

3. Create simple model that tries to predict which clients will contribute over $1,000,000 in production in 9 months based on Complexity data only. Is there a "formula" for complexity that indicates that the client WILL, WON'T ever contribute significant production?

4. Explore link between Complexity Points and # of Meetings, Zooms, calls in subsequent 9 months.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTEN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import mca
import prince
from IPython.display import display

In [ ]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Read in excel spreadsheets
complexity = pd.read_excel('../data/RohitCapstoneDataApr2022.xlsx', sheet_name = 'Complexity Data') \
    .rename(columns = {'rel_id': 'RelID', 'ComplexityRuleID': 'RuleID', 'ComplexityRuleType': 'RuleType'})

prod_outflow_columns = {
    'rel_id': 'RelID', 
    'CurrentRelValue': 'CurrentValue',
    'Beginning Rel Value': 'BeginningValue',
    'ThreeMonthProd': '3MonthProd',
    'SixMonthProd': '6MonthProd',
    'NineMonthProd': '9MonthProd',
    'ThreeMonthOutflow': '3MonthOutflow',
    'SixMonthOutflow': '6MonthOutflow',
    'NineMonthOutflow': '9MonthOutflow'
}
prod_outflow = pd.read_excel('../data/RohitCapstoneDataApr2022.xlsx', sheet_name = 'Prod-Outflow Data', skiprows = 1) \
    .rename(columns = prod_outflow_columns)[prod_outflow_columns.values()]

other = pd.read_excel('../data/RohitCapstoneDataApr2022.xlsx', sheet_name = 'Other Data as of 202105') \
    .drop(columns = 'AsOf')

meetings = pd.read_excel('../data/RohitCapstoneDataApr2022.xlsx', sheet_name = 'Subsequent Mtgs', converters = {'AsOf': str})
date_filter = ['202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204']
meetings = meetings.loc[meetings['AsOf'].isin(date_filter)]
meetings['Year'] = meetings['AsOf'].str[0:4]
meetings['Month'] = meetings['AsOf'].str[4:6]
meetings['Interactions'] = meetings['Call'] + meetings['Meeting'] + meetings['Zoom']

In [ ]:
# Show complexity rules
complexity_table = complexity[['RuleID', 'RuleType', '# Pts', 'Name']].sort_values('RuleID').drop_duplicates().reset_index(drop = True)
complexity_table = complexity_table.merge(complexity.value_counts('RuleID').to_frame('# RelIDs'), on = 'RuleID')
complexity_table
# complexity_table.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

In [ ]:
# Show meeting categories
meetings[['CategoryName']].drop_duplicates().reset_index(drop = True)

In [ ]:
# Group non-client review meetings together 
meetings.loc[meetings['CategoryName'] != 'Client Review', 'CategoryName'] = 'Everything Else'

# Sum up interactions by year, month, and meeting type 
meetings.loc[meetings['CategoryName'] == 'Client Review'].groupby(['Year', 'Month', 'CategoryName'], as_index = False)['Interactions'].sum()

In [ ]:
# Summarize accounts by BeginningValue
print('Total Accounts: ' + str(len(prod_outflow)))
print('Accounts with BeginningValue less than $500,000: ' + str(sum(prod_outflow['BeginningValue'] < 500000)))
print('Accounts with BeginningValue less than $250,000: ' + str(sum(prod_outflow['BeginningValue'] < 250000)))
print('Accounts with BeginningValue less than $100,000: ' + str(sum(prod_outflow['BeginningValue'] < 100000)))
print('Accounts with BeginningValue equal to $0: ' + str(sum(prod_outflow['BeginningValue'] == 0)))
print('Accounts with BeginningValue less than $0: ' + str(sum(prod_outflow['BeginningValue'] < 0)))

In [ ]:
# Filter accounts with BeginningValue less than $500,000
prod_outflow = prod_outflow[prod_outflow['BeginningValue'] >= 500000]

# Calculate net, net percentage, and percentage change variables
prod_outflow['3MonthNet'] = prod_outflow['3MonthProd'] + prod_outflow['3MonthOutflow']
prod_outflow['6MonthNet'] = prod_outflow['6MonthProd'] + prod_outflow['6MonthOutflow']
prod_outflow['9MonthNet'] = prod_outflow['9MonthProd'] + prod_outflow['9MonthOutflow']

prod_outflow['3MonthNetPct'] = prod_outflow['3MonthNet'] / prod_outflow['BeginningValue'] * 100
prod_outflow['6MonthNetPct'] = prod_outflow['6MonthNet'] / prod_outflow['BeginningValue'] * 100
prod_outflow['9MonthNetPct'] = prod_outflow['9MonthNet'] / prod_outflow['BeginningValue'] * 100

In [ ]:
# Summarize accounts by 9MonthProd
print('Total Accounts: ' + str(len(prod_outflow)))
print('Accounts with Production >= $1M: ' + str(sum(prod_outflow['9MonthProd'] >= 1000000)))
print('Accounts with Production between $500k-$1M: ' + str(sum((prod_outflow['9MonthProd'] >= 500000) & (prod_outflow['9MonthProd'] < 1000000))))
print('Accounts with Production between $250k-$500k: ' + str(sum((prod_outflow['9MonthProd'] >= 250000) & (prod_outflow['9MonthProd'] < 500000))))
print('Accounts with Production between $100k-$250k: ' + str(sum((prod_outflow['9MonthProd'] >= 100000) & (prod_outflow['9MonthProd'] < 250000))))
print('Accounts with Production between $50k-$100k: ' + str(sum((prod_outflow['9MonthProd'] >= 50000) & (prod_outflow['9MonthProd'] < 100000))))
print('Accounts with Production between $0-$50k: ' + str(sum((prod_outflow['9MonthProd'] > 0) & (prod_outflow['9MonthProd'] < 50000))))
print('Accounts with Production == $0: ' + str(sum(prod_outflow['9MonthProd'] == 0)))
print('Accounts with Production < $0: ' + str(sum(prod_outflow['9MonthProd'] < 0)))

In [ ]:
# Sum up complexity points per RelID
complexity_pts = complexity.groupby('RelID', as_index = False)['# Pts'].sum()

# Sum up client interactions per RelID
interactions = meetings.loc[meetings['CategoryName'] == 'Client Review'].groupby(['RelID'], as_index = False)['Interactions'].sum()

# Merge complexity_pts, prod_outflow, other, and interactions data
pts_prod_outflow = complexity_pts.merge(prod_outflow, on = 'RelID').set_index('RelID')
pts_prod_outflow_other = pts_prod_outflow.merge(other, on = 'RelID').set_index('RelID')
pts_prod_outflow_other_interactions = pts_prod_outflow_other.merge(interactions, how = 'left', on = 'RelID').set_index('RelID').fillna(0)

In [ ]:
# Sum up complexity rule dummy variables per RelID
complexity_rules = pd.get_dummies(complexity.set_index('RelID')['RuleID'].astype(str)).groupby('RelID').sum().reset_index()

# Create ordinal complexity rule variables
complexity_rules['2-3'] = complexity_rules['2'] * 1 + complexity_rules['3'] * 2
complexity_rules['4-5'] = complexity_rules['4'] * 1 + complexity_rules['5'] * 2
complexity_rules['6-7'] = complexity_rules['6'] * 1 + complexity_rules['7'] * 2
complexity_rules['8-9'] = complexity_rules['8'] * 1 + complexity_rules['9'] * 2
complexity_rules['10-13'] = complexity_rules['10'] * 1 + complexity_rules['11'] * 2 + complexity_rules['12'] * 3 + complexity_rules['13'] * 4
complexity_rules['14-16'] = complexity_rules['14'] * 1 + complexity_rules['15'] * 2 + complexity_rules['16'] * 3
complexity_rules['41-43'] = complexity_rules['41'] * 1 + complexity_rules['42'] * 2 + complexity_rules['43'] * 3
complexity_rules['49-51'] = complexity_rules['49'] * 1 + complexity_rules['50'] * 2 + complexity_rules['51'] * 3
complexity_rules['52-53'] = complexity_rules['52'] * 1 + complexity_rules['53'] * 2
complexity_rules['65'] = 1 - complexity_rules[['4', '5', '60']].sum(axis = 1)

# Merge complexity_rules, prod_outflow, other, and interactions data
rules_prod_outflow = complexity_rules.merge(prod_outflow, on = 'RelID').set_index('RelID')
rules_prod_outflow_other = rules_prod_outflow.merge(other, on = 'RelID').set_index('RelID')
rules_prod_outflow_other_interactions = rules_prod_outflow_other.merge(interactions, how = 'left', on = 'RelID').set_index('RelID').fillna(0)

In [ ]:
# Define feature sets
all_rules = [
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
    '16', '17', '18', '19', '20', '21', '22', '23', '24', '26', '27', '28', '29', '30',
    '31', '32', '33', '37', '38', '39', '40', '41', '42', '43', '45', '49', '50', '51',
    '52', '53', '54', '55', '56', '57', '58', '60', '62', '63', '64'
]

collapsed_rules = [
    '1', '2-3', '4-5', '6-7', '8-9', '10-13', '14-16', '17', '18', '19',
    '20', '21', '22', '23', '24', '26', '27', '28', '29', '30', '31', 
    '32', '33', '37', '38', '39', '40', '41-43', '45', '49-51', '52-53',
    '54', '55', '56', '57', '58', '62', '63', '65'
]

other_cols = [
    'CurrentValue', 'BeginningValue', '3MonthProd', '6MonthProd', '9MonthProd',
    '3MonthOutflow', '6MonthOutflow', '9MonthOutflow', '3MonthNet', '6MonthNet', 
    '9MonthNet', '3MonthNetPct', '6MonthNetPct', '9MonthNetPct', 'TotalUnmanagedValue',
    'TotalCashValue', 'TotalCashUnmanagedValue', '# Accts', 'Interactions'
]

In [ ]:
# Generate crosstab heatmap of complexity points vs. client interactions
pts_interactions = pts_prod_outflow_other_interactions[['# Pts', 'Interactions']].copy()

pts_interactions.loc[pts_interactions['# Pts'] <= 3, 'Complexity Points'] = '0-3'
pts_interactions.loc[pts_interactions['# Pts'].between(4, 6), 'Complexity Points'] = '4-6'
pts_interactions.loc[pts_interactions['# Pts'].between(7, 9), 'Complexity Points'] = '7-9'
pts_interactions.loc[pts_interactions['# Pts'].between(10, 13), 'Complexity Points'] = '10-13'
pts_interactions.loc[pts_interactions['# Pts'] >= 14, 'Complexity Points'] = '14+'

pts_interactions.loc[pts_interactions['Interactions'] == 0, 'Client Interactions'] = '0'
pts_interactions.loc[pts_interactions['Interactions'] == 1, 'Client Interactions'] = '1'
pts_interactions.loc[pts_interactions['Interactions'].between(2,3), 'Client Interactions'] = '2-3'
pts_interactions.loc[pts_interactions['Interactions'] >= 4, 'Client Interactions'] = '4+'

pts_interactions_crosstab = pd.crosstab(pts_interactions['Complexity Points'], pts_interactions['Client Interactions'])
pts_interactions_crosstab = pts_interactions_crosstab.reindex(['14+', '10-13', '7-9', '4-6', '0-3'], axis = 'rows')

plt.figure(figsize = (7, 6), dpi = 300)
sns.heatmap(pts_interactions_crosstab, annot= True, fmt='d', cmap = plt.cm.RdBu_r)
plt.show()

In [ ]:
# Generate pearson correlation heatmap of total complexity points vs. other variables
# pearson_corr = pts_prod_outflow_other.corr('pearson')
# mask = np.triu(np.ones_like(pearson_corr, dtype = bool))

# plt.figure(figsize = (20, 20), dpi = 300)
# sns.heatmap(pearson_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
# plt.title('Pearson Correlation - Total Complexity Points')
# plt.show()

In [ ]:
# Generate spearman correlation heatmap of total complexity points vs. other variables
spearman_corr = pts_prod_outflow_other_interactions.corr('spearman')
mask = np.triu(np.ones_like(spearman_corr, dtype = bool))

plt.figure(figsize = (20, 20), dpi = 300)
sns.heatmap(spearman_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
plt.title('Spearman Correlation - Total Complexity Points')
plt.show()

In [ ]:
# Generate scatterplots of 9MonthNet and 9MonthNetPct vs. # Pts
fig, axs = plt.subplots(1, 2, figsize = (12, 5), dpi = 300)

sns.scatterplot(x = pts_prod_outflow_other['# Pts'], 
                y = pts_prod_outflow_other['9MonthNet'], 
                s = 8, 
                ax = axs[0])

sns.scatterplot(x = pts_prod_outflow_other['# Pts'], 
                y = pts_prod_outflow_other['9MonthNetPct'], 
                s = 8, 
                ax = axs[1])
axs[1].set(ylim=(-1000, 1000))

plt.show()

In [ ]:
# Generate pearson correlation heatmap of all complexity rules vs. other variables
# pearson_corr = rules_prod_outflow_other_interactions[all_rules + other_cols].corr('pearson')
# mask = np.triu(np.ones_like(pearson_corr, dtype = bool))

# plt.figure(figsize = (32, 32), dpi = 300)
# sns.heatmap(pearson_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
# plt.title('Pearson Correlation - All Complexity Rules')
# plt.show()

In [ ]:
# Generate spearman correlation heatmap of all complexity rules vs. other variables
spearman_corr = rules_prod_outflow_other_interactions[all_rules + other_cols].corr('spearman')
mask = np.triu(np.ones_like(spearman_corr, dtype = bool))

plt.figure(figsize = (32, 32), dpi = 300)
sns.heatmap(spearman_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
plt.title('Spearman Correlation - All Complexity Rules')
plt.show()

In [ ]:
# Generate pearson correlation heatmap of collapsed complexity rules vs. other variables
# pearson_corr = rules_prod_outflow_other_interactions[collapsed_rules + other_cols].corr('pearson')
# mask = np.triu(np.ones_like(pearson_corr, dtype = bool))

# plt.figure(figsize = (32, 32), dpi = 300)
# sns.heatmap(pearson_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
# plt.title('Pearson Correlation - Collapsed Complexity Rules')
# plt.show()

In [ ]:
# Generate spearman correlation heatmap of collapsed complexity rules vs. other variables
spearman_corr = rules_prod_outflow_other_interactions[collapsed_rules + other_cols].corr('spearman')
mask = np.triu(np.ones_like(spearman_corr, dtype = bool))

plt.figure(figsize = (32, 32), dpi = 300)
sns.heatmap(spearman_corr, mask = mask, annot = True, fmt = '.2f', cmap = plt.cm.RdBu_r, cbar = False)
plt.title('Spearman Correlation - Collapsed Complexity Rules')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6), dpi = 300)
mc = prince.MCA(n_components = 2).fit(rules_prod_outflow_other_interactions[all_rules])
ax.set_xlabel('Component 1', fontsize = 12)
ax.set_ylabel('Component 2', fontsize = 12)
mc.plot_coordinates(
    X = rules_prod_outflow_other_interactions[all_rules], 
    row_points_alpha = 0.02, 
    row_points_size = 3, 
    show_column_labels = True, 
    legend_n_cols = 4, 
    ax = ax
)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6), dpi = 300)
mc = prince.MCA(n_components = 2).fit(rules_prod_outflow_other_interactions.loc[~(rules_prod_outflow_other_interactions[['28','29','33']].sum(axis = 1) > 0), [rule for rule in all_rules if rule not in ['28', '29', '33']]])
ax.set_xlabel('Component 1', fontsize = 12)
ax.set_ylabel('Component 2', fontsize = 12)
mc.plot_coordinates(
    X = rules_prod_outflow_other_interactions.loc[~(rules_prod_outflow_other_interactions[['28','29','33']].sum(axis = 1) > 0), [rule for rule in all_rules if rule not in ['28', '29', '33']]], 
    row_points_alpha = 0.02, 
    row_points_size = 3, 
    show_column_labels = True, 
    legend_n_cols = 5, 
    ax = ax
)
plt.show()

In [ ]:
K = 54
Theta = (K / (K - 1.)) * (np.sum(np.square(mc.s_)**2))
print(Theta)

In [ ]:
K = 54
Theta = (K / (K - 1.)) * (np.sum(np.square(mc.s_)))
print(Theta)

In [ ]:
distances = pdist(rules_prod_outflow_other_interactions[all_rules], metric = 'hamming')
mergings = linkage(distances, method = 'complete')

fig, ax = plt.subplots(figsize = (12, 4), dpi = 300)
dendrogram(mergings,
           no_labels = True,
           leaf_rotation = 90,
           leaf_font_size = 6,
           ax = ax)
plt.show()

In [ ]:
distances = pdist(rules_prod_outflow_other_interactions[all_rules], metric = 'cosine')
mergings = linkage(distances, method = 'complete')

fig, ax = plt.subplots(figsize = (12, 4), dpi = 300)
dendrogram(mergings,
           no_labels = True,
           leaf_rotation = 90,
           leaf_font_size = 6,
           ax = ax)
plt.show()

In [ ]:
distances = pdist(rules_prod_outflow_other_interactions[all_rules].T, metric = 'hamming')
mergings = linkage(distances, method = 'complete')

fig, ax = plt.subplots(figsize = (12, 4), dpi = 300)
dendrogram(mergings, 
           labels = rules_prod_outflow_other_interactions[all_rules].columns, 
           leaf_rotation = 90, 
           leaf_font_size = 6, 
           ax = ax)
ax.tick_params(axis='x', which='major', labelsize = 10)
plt.show()

In [ ]:
distances = pdist(rules_prod_outflow_other_interactions[all_rules].T, metric = 'cosine')
mergings = linkage(distances, method = 'complete')

fig, ax = plt.subplots(figsize = (12, 4), dpi = 300)
dendrogram(mergings, 
           labels = rules_prod_outflow_other_interactions[all_rules].columns, 
           leaf_rotation = 90, 
           leaf_font_size = 6, 
           ax = ax)
ax.tick_params(axis = 'x', which = 'major', labelsize = 10)
plt.show()

In [ ]:
cluster = AgglomerativeClustering(affinity = 'precomputed', 
                                  linkage = 'complete',
                                  n_clusters = 5).fit(squareform(distances))

In [ ]:
silhouette_score(X = squareform(distances), labels = cluster.labels_, metric = 'precomputed')

In [ ]:
max_clusters = 20

silhouette_scores = []

for n_clusters in range(2, max_clusters + 1):
    cluster = AgglomerativeClustering(affinity = 'precomputed', 
                                      linkage = 'complete',
                                      n_clusters = n_clusters).fit(squareform(distances))
    
    silhouette_scores.append(silhouette_score(squareform(distances), cluster.labels_, metric = 'precomputed'))

In [ ]:
plt.figure(figsize = (10,6))

plt.plot(range(2, max_clusters + 1), silhouette_scores)